# 欠勤率の予測

## ライブラリのインポート

In [ ]:
# 関連するライブラリをインポートします
import pandas as pd
import numpy as np

## データの読み込み

In [ ]:
# 前処理をしたCSVファイルの読み込み
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [ ]:
# 中身の確認
data_preprocessed.head()

## ターゲットの作成

In [ ]:
# 'Absenteeism Time in Hours'の中央値を見つける
data_preprocessed['Absenteeism Time in Hours'].median()

In [ ]:
# 上のコードで確認した中央値を元に、1と0に分けたターゲットを作成していきます

# レクチャーでのはじめのコードは以下の通りです
# targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 3, 1, 0)

# 中央値よりも大きいかどうかで1と0のターゲットを作成していきます
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [ ]:
# 中身の確認
targets

In [ ]:
# targetを変数に代入します
data_preprocessed['Excessive Absenteeism'] = targets

In [ ]:
# 前処理したデータの確認
data_preprocessed.head()

## ターゲットに関して

In [ ]:
# ターゲットがバランスしているか確認していきます
# targets.sum() は1の数を示しており
# the shape[0] はデータの数を示しています
targets.sum() / targets.shape[0]

In [ ]:
# 不要な列の削除
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis=1)

In [ ]:
# data_with_targets is data_preprocessed = Trueであれば、二つは同じオブジェクトであることを意味しています
# 二つのオブジェクトが異なっていれば、チェックポイントが作成されていることを示しています
data_with_targets is data_preprocessed

In [ ]:
# データの確認
data_with_targets.head()

## 入力データの作成

In [ ]:
data_with_targets.shape

In [ ]:
# 14番目までの列の抽出
data_with_targets.iloc[:,:14]

In [ ]:
# 最後の列を取り除く
data_with_targets.iloc[:,:-1]

In [ ]:
# 入力データを変数に格納する
unscaled_inputs = data_with_targets.iloc[:,:-1]

## データの標準化

In [ ]:
# 標準化するためのStandardScalerクラスをインポートします
from sklearn.preprocessing import StandardScaler

# クラスからオブジェクトを作成します
absenteeism_scaler = StandardScaler()

In [ ]:
# unscaled_inputsをモデルにフィットさせます
absenteeism_scaler.fit(unscaled_inputs)

In [ ]:
# transformメソッドを使って値を標準化させていきます
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [ ]:
# データの確認
scaled_inputs

In [ ]:
# 配列の形状の確認
scaled_inputs.shape

## データの分割

### ライブラリのインポート

In [ ]:
from sklearn.model_selection import train_test_split

### データの分割

In [ ]:
train_test_split(scaled_inputs, targets)

In [ ]:
# 訓練データとテストデータに関し、入力とターゲットに分けて変数に代入する
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [ ]:
# データの形状の確認
print (x_train.shape, y_train.shape)

In [ ]:
# データの形状の確認
print (x_test.shape, y_test.shape)

## sklearnを使ったロジスティック回帰モデルの作成

In [ ]:
# LogisticRegressionのインポート
from sklearn.linear_model import LogisticRegression

# 'metrics'モジュールのインポート
from sklearn import metrics

### モデルの訓練

In [ ]:
# LogisticRegressionクラスからオブジェクトを作成
reg = LogisticRegression()

In [ ]:
# モデルへのあてはめ
reg.fit(x_train,y_train)

In [ ]:
# モデルの正確性の計算
reg.score(x_train,y_train)

### 正確性の計算の実装

In [ ]:
# モデルに入れた結果の出力を変数に代入する
model_outputs = reg.predict(x_train)
model_outputs

In [ ]:
# ターゲットの確認
y_train

In [ ]:
# 出力とターゲットが同じかの確認
model_outputs == y_train

In [ ]:
# 出力をターゲットが同じである数のカウント
np.sum((model_outputs==y_train))

In [ ]:
# データの合計の数の確認
model_outputs.shape[0]

In [ ]:
# 出力とターゲットが同じ数を合計の数で割って正確性を求める
np.sum((model_outputs==y_train)) / model_outputs.shape[0]

### 折半と係数

In [ ]:
# 切片（バイアス）の確認
reg.intercept_

In [ ]:
# 係数（重み）の確認
reg.coef_

In [ ]:
# 列の名前の確認
unscaled_inputs.columns.values

In [ ]:
# 列の名前を変数に代入
feature_name = unscaled_inputs.columns.values

In [ ]:
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

In [ ]:
summary_table.index = summary_table.index + 1

summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

summary_table = summary_table.sort_index()
summary_table

## 係数の理解

In [ ]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [ ]:
summary_table

In [ ]:
summary_table.sort_values('Odds_ratio', ascending=False)

## モデルのテスト

In [ ]:
# モデルの正確性を求める
reg.score(x_test,y_test)

In [ ]:
# 確率の値を変数に代入します
predicted_proba = reg.predict_proba(x_test)

# 中身を確認します
predicted_proba

In [ ]:
predicted_proba.shape

In [ ]:
# 1に関連した確率を抽出します
predicted_proba[:,1]

## モデルの保存

In [ ]:
# モジュールのインポート
import pickle

In [ ]:
# モデルを保存します
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [ ]:
# scalerを保存します
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)